<a href="https://colab.research.google.com/github/Tyred/TimeSeries_OCC-PUL/blob/main/OC_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors

In [ ]:
path = 'drive/My Drive/UFSCar/FAPESP/IC/Data/UCRArchive_2018'

dataset = input('Dataset: ')
tr_data = np.genfromtxt(path + "/" + dataset + "/" + dataset + "_TRAIN.tsv", delimiter="\t",)
te_data = np.genfromtxt(path + "/" + dataset + "/" + dataset + "_TEST.tsv", delimiter="\t",)

In [ ]:
class_label = 2
train_data  = tr_data[tr_data[:, 0] == class_label, 1:] # train
test_data   = te_data[:, 1:]                            # test
labels      = te_data[:, 0]

print("Labels:", np.unique(labels))
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

In [ ]:
def OC_KNN(test_sample, k, threshold, distances_avg): # maybe J for OC-JKNN
  # New samples classification
  distances, indices = nbrs.kneighbors(test_sample.reshape(1,-1), k)
  avg = np.mean(distances)
  
  knbrs_sum = 0
  for idx in indices[0]:
    knbrs_sum += distances_avg[idx]
  
  knbrs_avg = knbrs_sum/k

  if avg/knbrs_avg <= threshold:
    return True
  else:
    return False

In [ ]:
# Fitting OC-KNN 
k = 1
threshold = 1

nbrs = NearestNeighbors(n_neighbors=k+1).fit(train_data)
distances, indices = nbrs.kneighbors(train_data)

distances_avg = []
for sample in distances:
  avg = np.mean(sample[1:])
  distances_avg.append(avg)

#OC_KNN(train_data, test_data[8233], k, threshold, distances_avg)

In [ ]:
target_class = np.where(labels==class_label)
negative_class = np.where(labels!=class_label)
print("Positive samples:", len(target_class[0]))
print("Negative samples:", len(negative_class[0]))

In [ ]:
tp = 0
for sample in test_data[target_class[0]]:
  if OC_KNN(train_data, sample, k, threshold, distances_avg):
    tp += 1

fn = len(target_class[0]) - tp

print("Recall:", tp/len(target_class[0]))

In [ ]:
tn = 0
for sample in test_data[negative_class[0]]:
  if not OC_KNN(train_data, sample, k, threshold, distances_avg):
    tn += 1

fp = len(negative_class[0]) - tn

print("Precision:", tp/(tp+fp))
print("Accuracy:", (tp+tn)/(tp+tn+fp+fn))